# 06 Train Test Split

In [1]:
import pandas as pd

In [2]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# widen the column width and overall display width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 0)

In [3]:
df: pd.DataFrame = pd.read_csv("../data/interim/data_encoded.csv", parse_dates=["time"], index_col="time")

In [4]:
df = df.sort_index()

In [5]:
# Define split point
split_index = int(len(df) * 0.8)

# Split
train_df = df.iloc[:split_index]
forecast_df = df.iloc[split_index:]

In [6]:
train_df.to_csv("../data/interim/train.csv", index=True)
forecast_df.to_csv("../data/interim/forecast.csv", index=True)

print(f"Train shape: {train_df.shape}, Forecast shape: {forecast_df.shape}")

Train shape: (112, 36), Forecast shape: (28, 36)
